In [1]:
import pamtra2
import numpy as np
import pandas as pn
import xarray as xr
from collections import OrderedDict
from copy import deepcopy, copy
import toolz

In [2]:
additionalDims = OrderedDict()
additionalDims['time'] = pn.date_range('2016-01-01','2016-01-05',freq='D')[:1]
additionalDims['lat'] = np.arange(70,80)
nHeights = 100

pam2 = pamtra2.pamtra2(nHeights,additionalDims = additionalDims)



In [3]:

pam2.profile.height[:] = np.linspace(0,1000,nHeights)
pam2.profile.temperature[:] = 250 
pam2.profile.relativeHumidity[:] = 90
pam2.profile.pressure[:] = 10000



In [4]:
pam2.profile.hydrometeorWaterContent
#Array exists, but length of hydrometeor coordinate ==0


<xarray.DataArray 'hydrometeorWaterContent' (time: 1, lat: 10, layer: 100, hydrometeor: 0)>
array([], shape=(1, 10, 100, 0), dtype=float64)
Coordinates:
  * time         (time) datetime64[ns] 2016-01-01
  * lat          (lat) int64 70 71 72 73 74 75 76 77 78 79
  * layer        (layer) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
  * hydrometeor  (hydrometeor) |S128 
Attributes:
    unit:     kg/m^3

In [5]:
# # OR EASIER if desired: 
# pam2 = pamtra2.importers.profiles.usStandardAtmosphere(heigths)

In [6]:

pam2.addHydrometeor(
    name = 'rain', #or None, then str(index)
    kind = 'liquid', #liquid, ice
    nBins =40,
    sizeCenter = (
        pamtra2.hydrometeors.maximumDimension.linspace, 
        {
            'Dmin' : 1e-6,
            'Dmax' : 1e-2,
        }
    ),
    sizeDistribution = (
        pamtra2.hydrometeors.sizeDistribution.exponentialN0WC, 
        {
            'N0':   8e6,
            'WC' : pam2.profile.hydrometeorWaterContent, 
        },
    ),
    aspectRatio = 1.0,
    mass = (
        pamtra2.hydrometeors.mass.ellipsoid,
    ),
    density = (
        pamtra2.hydrometeors.density.water,
    ),
    crossSectionArea = (
        pamtra2.hydrometeors.crossSectionArea.sphere,
    ),
)



/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2/core.py:94: FutureWarning: xarray.DataArray.__contains__ currently checks membership in DataArray.coords, but in xarray v0.11 will change to check membership in array values.
  if name in self.profile.hydrometeor:


In [7]:
pam2.addHydrometeor(
    name='snow',
    nBins = 20,
    sizeCenter=(
        pamtra2.hydrometeors.maximumDimension.logspace, #function/object to call for getting sizes
        {
            'Dmin' : 1e-6,
            'Dmax' : 1e-2,
        }
    ),
    sizeDistribution = (
        pamtra2.hydrometeors.sizeDistribution.exponentialFieldWC, #function/object to call for getting psd
        {
            'temperature' : pam2.profile.temperature,
            'WC' : pam2.profile.hydrometeorWaterContent, #test whether hydrodim is included! for all input vars!
            'massSizeA' : 0.0121, #mass size relation required to estimate exponential doistribution from N0
            'massSizeB' : 1.9,
        },
    ),
    aspectRatio = (
        0.6,
    ),
    crossSectionArea = (
        pamtra2.hydrometeors.crossSectionArea.powerLaw, #function/object to call for getting crossSectionArea
        {
            'areaSizeA' : 0.01,
            'areaSizeB' : 1.8,
        },
    ),
    mass = (
        pamtra2.hydrometeors.mass.powerLaw, #function/object to call for getting mass
        {
            'massSizeA' : 0.0121,
            'massSizeB' : 1.9,
        },
    ),
    density = (
        pamtra2.hydrometeors.density.softEllipsoid, #function/object to call for getting density
        {
            'minDensity' : 100,
        },
    ),
)



/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2/core.py:94: FutureWarning: xarray.DataArray.__contains__ currently checks membership in DataArray.coords, but in xarray v0.11 will change to check membership in array values.
  if name in self.profile.hydrometeor:


In [8]:
pam2.profile.hydrometeorWaterContent.values[:] = 0
pam2.profile.hydrometeorWaterContent.values[:,:,20:40] = 1e-4



In [9]:
pam2.hydrometeors['rain'].calculateProperties()

callable
callable
not callable 1.0
callable
NOT softsphere!
not callable (1000.0,)
callable


/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2/hydrometeors/sizeDistribution.py:145: RuntimeWarning: divide by zero encountered in true_divide
  lambd = (massSizeA * N0 * scipy.special.gamma(massSizeB+1.) / WC)**(1. /(massSizeB+1.))


<xarray.Dataset>
Dimensions:                   (frequency: 0, lat: 10, layer: 100, sizeBin: 40, time: 1)
Coordinates:
  * time                      (time) datetime64[ns] 2016-01-01
  * lat                       (lat) int64 70 71 72 73 74 75 76 77 78 79
  * layer                     (layer) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 ...
  * sizeBin                   (sizeBin) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 ...
  * frequency                 (frequency) float64 
Data variables:
    sizeCenter                (time, lat, layer, sizeBin) float64 1e-06 ...
    particleSizeDistribution  (time, lat, layer, sizeBin) float64 0.0 0.0 ...
    aspectRatio               (time, lat, layer, sizeBin) float64 1.0 1.0 ...
    crossSectionArea          (time, lat, layer, sizeBin) float64 7.854e-25 ...
    mass                      (time, lat, layer, sizeBin) float64 5.236e-16 ...
    density                   (time, lat, layer, sizeBin) float64 1e+03 ...

In [10]:
%debug

> /Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2/hydrometeors/core.py(169)_arrayOrFunc()
    167             #inspect function to get the required arguments
    168             funcArgs, funcVarargs, funcKeywords, funcDefaults = inspect.getargspec(func)
--> 169             funcDefaults = dict(zip(funcArgs[-len(funcDefaults):],funcDefaults))
    170 
    171             #where do we find the required data?

ipdb> funcArgs[-len(funcDefaults):]
*** TypeError: object of type 'NoneType' has no len()
ipdb> funcArgs
['sizeCenter']
ipdb> len(funcDefaults)
*** TypeError: object of type 'NoneType' has no len()
ipdb> funcDefaults
ipdb> funcDefaults is None
True
ipdb> q


In [12]:
pam2.hydrometeors['snow'].calculateProperties()

callable
callable
not callable (0.6,)
callable
softsphere!
callable
callable


/Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pamtra2/hydrometeors/sizeDistribution.py:145: RuntimeWarning: divide by zero encountered in true_divide
  lambd = (massSizeA * N0 * scipy.special.gamma(massSizeB+1.) / WC)**(1. /(massSizeB+1.))


<xarray.Dataset>
Dimensions:                   (lat: 10, layer: 100, sizeBin: 20, time: 1)
Coordinates:
  * time                      (time) datetime64[ns] 2016-01-01
  * lat                       (lat) int64 70 71 72 73 74 75 76 77 78 79
  * layer                     (layer) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 ...
  * sizeBin                   (sizeBin) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 ...
Data variables:
    maximumDimension          (time, lat, layer, sizeBin) float64 1e-06 ...
    particleSizeDistribution  (time, lat, layer, sizeBin) float64 0.0 0.0 ...
    aspectRatio               (time, lat, layer, sizeBin) float64 0.6 0.6 ...
    crossSectionArea          (time, lat, layer, sizeBin) float64 1.585e-13 ...
    mass                      (time, lat, layer, sizeBin) float64 4.817e-14 ...
    density                   (time, lat, layer, sizeBin) float64 917.0 ...

In [ ]:
pam2.hydrometeors['ice'].addScatteringProperties(
    refractiveIndex = (
        pamtra2.refractiveIndex.snow, 
        funcArgs = {
            'model_mix':'Bruggeman',
            'model_ice':'Matzler_2006',
            'densities':
        }
    ),
    singleScattering = (
        pamtra2.singleScattering.rayleighGans, 
        funcArgs = {
        },
    ),
)



In [16]:

#this function will add hydrometeor confihuration to pam2.hydrometeors and 
#append hydrometeor to coordinates of e.g. pam2.profile.lwc
#Note that only the configuration will stored! All the functions
#will be calles later by the forward model! 
pam2.addHydrometeor(
    name = 'snow1', #or None, then str(index)
    index = 0, #or None, then append
    kind = 'snow', #liquid, ice
    nBins = 80,
    sizes = (
        pamtra2.hydrometeors.sizes.logspace, #function/object to call for getting sizes
        funcArgs = {
            Dmin = 1e-6,
            Dmax = 1e-4,
        }
    ),
    psd = (
        pamtra2.hydrometeors.sizeDistributions.exponentialFieldLwc, #function/object to call for getting psd
        funcArgs = {
            temperature : pam2.profile.temperature,
            lwc : pam2.profile.lwc, #test whether hydrodim is included! for all input vars!
            massSizeA : 0.01, #mass size relation required to estimate exponential doistribution from N0
            massSizeB : 1.8,
        },
    ),
    aspectRatio = (
        0.6,
    ),
    mass = (
        pamtra2.hydrometeors.mass.powerLaw, #function/object to call for getting mass
        funcArgs = {
            massSizeA : 0.01,
            massSizeB : 1.8,
        },
    ),
    density = (
        pamtra2.hydrometeors.density.softSphere, #function/object to call for getting density
        funcArgs = {
            minDensity : 100,
#             maxDensity : , defaults to ice!
        },
    ),
    crossSectionArea = (
        pamtra2.hydrometeors.area.powerLaw, #function/object to call for getting crossSectionArea
        funcArgs = {
            areaSizeA : 0.01,
            areaSizeB : 1.8,
        },
    ),
)
#for example, pamtra2.hydrometeors.area.powerLaw will be an object
# which will be called with pamtra2.hydrometeors.area.powerLaw(diameterCenter,
# areaSizeA=areaSizeA,areaSizeB=areaSizeB)



pam2.addHydrometeor(
    name = 'rain1', #or None, then str(index)
    index = 1, #or None, then append
    kind = 'liquid', #liquid, ice
    nBins =40,
    sizes = (
        pamtra2.hydrometeors.sizes.linspace, 
        funcArgs = {
            Dmin = 1e-6,
            Dmax = 1e-4,
        }
    ),
    psd = (
        pamtra2.hydrometeors.sizeDistributions.exponentialLwc, #function/object to call for getting psd
        funcArgs = {
            lwc : pam2.profile.lwc, #test whether hydrodim is included! for all input vars!
            lambd : 10,
        },
    ),
    aspectRatio = (
        pamtra2.hydrometeors.aspectRatio.rainAspectRatioModel,
        funcArgs = {
            sampleKeyword : 10, 
        },
    ),
    mass = (
        pamtra2.hydrometeors.mass.ellipsoid,
    ),
    density = (
        pamtra2.hydrometeors.density.water, # or 1000.
    ),
    crossSectionArea = (
        pamtra2.hydrometeors.area.ellipsoid,
    ),
)

#in case we already have the full spectrum!

pam2.addHydrometeor(
    name = 'ice1', #or None, then str(index)
    index = 1, #or None, then append
    kind = 'liquid', #liquid, ice
    nBins =2,
    sizes = (
        pamtra2.hydrometeors.sizes.monodispers, 
        funcArgs = {
            Dmono = 1e-5,
        },
    ),
    psd = (
        pamtra2.profiles.hydrometeor_size_distribution, #array with discrete n(D) exists, but has len ==0.
    ),
    aspectRatio = (
        1.0,
    ),
    mass = (
        pamtra2.hydrometeors.mass.ellipsoid,
    ),
    density = (
        pamtra2.hydrometeors.density.ice, # or 971.
    ),
    crossSectionArea = (
        pamtra2.hydrometeors.area.ellipsoid,
    ),
)
#now the array is prepared because addHydrometeor got an array as an argumetn for psd
pam2.profiles.hydrometeor_size_distribution.sel(hydrometeor='ice1')[:] = ICE_PSD_FROM_AIRCRAFT


#now also profile.lwc is prepare because we know how many hydrometeors we have:
pam2.profile.lwc[:] = 0.1


# OR EASIER for all of the above: 
pamtra2.importer.profiles.CosmoColumsNetcdf('COSMO_column_20170102.nc') #will take care of hydrometeors and profiles


#kazr is a copy of the pam object with added information about the instrument
kazr = pam2.createInstrument(
    name= 'KAZR'
    kind = 'dopplerRadar',
    #the pamtra2.forwardOperators.spectralRadarSimulator class will contain all the magic and create the object to be retruend
    method = pamtra2.forwardOperators.spectralRadarSimulator, 
    frequencies = [35],
    settings = {
        'nyquistVMax' : 10,
        'nyquistVMin' : -10,
        'nFFT' : 512,
        'K2': 0.92,
        ...
    }
)


#add information about scattering properties
kazr.setHydrometeorScattering(
    'rain1',
    refractiveIndex = (
        pamtra2.refractiveIndex.water, 
        funcArgs = {
            'model' : 'Turner',
        }
    ),
    singleScattering = (
        pamtra2.singleScattering.tmatrix, 
        funcArgs = {
            'cached' : True,
        },
    ),
)

kazr.setHydrometeorScattering(
    'snow1', 
    refractiveIndex = (
        pamtra2.refractiveIndex.snow, 
        funcArgs = {
            'model_mix':'Bruggeman',
            'model_ice':'Matzler_2006',
        }
    ),
    singleScattering = (
        pamtra2.singleScattering.rayleighGans, 
        funcArgs = {
        },
    ),
)

kazr.setHydrometeorScattering(
    'ice1',
    # refractiveIndex = (), For missing descriptions, default values 
    # will be used depending on particle kind!
    # singleScattering = (
    #     pamtra2.singleScattering.mie, 
    # ),
)


hatpro = pam2.createInstrument(
    name= 'Hatpro'
    kind = 'MWR',
    #pamtra2.forwardOperators.RT4 will share a lot of code with pamtra2.forwardOperators.spectralRadarSimulator
    method = pamtra2.forwardOperators.RT4, 
    frequencies = [22.24, 23.04, 23.84, 25.44, 26.24, 27.84, 31.40],
    settings = {
        'bandWidths' : [0.230, 0.230, 0.230, 0.230, 0.230, 0.230, 0.230],
        ....
    }
)

#We can either copy the scattering properties or do new ones!
hatpro.setHydrometeorScattering(
...
)



ceilo = pam2.createInstrument(
    name= 'Ceilo'
    kind = 'Ceilometer',
    method = pamtra2.forwardOperators.Ceilosim, 
    wavelengths = [905],
    settings = {
        'property' : 10,
        ...
    }
)

ceilo.setHydrometeorScattering(
...
)

# OR EASIER: 
WSACR = pamtra.importer.instruments.WSACR(site='Oliktok Point',configuration='20171004')
WSACR.setHydrometeorScattering(
...
)



kazr.run() #this command runs all the functions defined above!
kazr.results.to_netcdf('kazr.nc')

hatpro.run()
hatpro.results.to_netcdf('hatpro.nc')

ceilo.run()
ceilo.results.to_netcdf('ceilo.nc')

SyntaxError: invalid syntax (<ipython-input-16-98e892c92325>, line 13)

In [ ]:
%debug

> /Volumes/User/mmaahn/anaconda/envs/python3/lib/python3.6/site-packages/pyPamtra2/hydrometeors/core.py(139)_arrayOrFunc()
    137 
    138         thisProperty = np.zeros(thisShape) * np.nan
--> 139         if callable(thisDesription[0]):
    140             print('callable')
    141             func, kwargs = thisDesription

ipdb> thisDesription
1.0
ipdb> thisDesription
1.0
ipdb> type(thisDesription
*** SyntaxError: unexpected EOF while parsing
ipdb> type(thisDesription)
<class 'float'>


In [53]:
np.array(list(map(lambda x,y:x+y**2,np.random.random((10,10)),np.random.random((10,10)))))

(10, 10)

In [16]:
np.asarray(pam2.profile.temperature)

array([[[ 250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,
          250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,
          250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,
          250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,
          250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,
          250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,
          250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,
          250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,
          250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,
          250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,
          250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,
          250.],
        [ 250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,
          250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,  250.,
          250.,  250.,  250.,  250.,  250.,  25

In [31]:
hyd.discreteProperties.aspectRatio.max()

<xarray.DataArray 'aspectRatio' ()>
array(90427118.89111808)

In [31]:
import numba

@numba.jit(nopython=False)
def modifiedGamma(sizeCenter,N0,lambd,mu=0,gamma=1):
  """
  classical modifed gamma distribution

  Parameters
  ----------
  sizeCenter : array_like
    particle size at center of size bin
  N0 : array_like
    N0 prefactor (default None)
  lambd : float or array_like
    lambda parameter (default array)
  mu : float or array_like
    mu parameter (default array)
  gamma : float or array_like
    gamma parameter (default array)

  Returns
  -------

  N : array
    particle size distribution with shape = N0.shape + sizeCenter.shape
  """

  N = N0 * sizeCenter**mu * np.exp(-lambd * sizeCenter**gamma)

  return N

In [32]:



sizeCenter=np.zeros((1000,10,4))
sizeCenter[:] = np.logspace(-6,-2,4)
# sizeCenter =np.logspace(-6,-2,100).reshape((1000,10))
N0 = np.random.random((1000,10))
lambd = np.random.random((1000,10))

# vecFunc = np.vectorize(lambda sizeCenter,N0,lambd : modifiedGamma(sizeCenter,N0,lambd))


%timeit modifiedGamma(sizeCenter.T,N0.T,lambd.T).T

784 µs ± 13.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
